<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">

### Consigna del desafío 2

**Cada experimento realizado debe estar acompañado de una explicación o interpretación de lo observado**

Recuerden que su notebook de entrega debe poder correrse de inicio a fin sin la aparición de errores.

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro artista del dataset Songs.
- Elegir términos de interés y buscar términos más similares y menos similares.
- Realizar una reduccion de dimensionalidad a los embeddings, llevándolos a 2 dimensiones. Graficar los embeddings proyectados y seleccionar una cantidad de términos (variable MAX_WORDS) de forma tal que la visualización sea adecuada.
- Inspeccionar el grafico y buscar pequeños grupos de palabras que puedan formarse. Interpretarlos e intentar obtener conclusiones. En lo posible, acompañar los grupos de palabras con capturas (y pegarlas en celdas de texto)

In [66]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
try:
  from gensim.models import Word2Vec
except:
  !pip install gensim
  from gensim.models import Word2Vec

### Cargar Dataset

In [67]:
# Descargar la carpeta de dataset
import os
import platform
if os.access('./songs_dataset', os.F_OK) is False:
    if os.access('songs_dataset.zip', os.F_OK) is False:
        if platform.system() == 'Windows':
            !curl https://raw.githubusercontent.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/main/datasets/songs_dataset.zip -o songs_dataset.zip
        else:
            !wget songs_dataset.zip https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/datasets/songs_dataset.zip
    !unzip -q songs_dataset.zip
else:
    print("El dataset ya se encuentra descargado")

El dataset ya se encuentra descargado


### Leer songs_dataset de Bruno Mars

In [68]:
df = pd.read_csv('songs_dataset/bruno-mars.txt', sep='/n', header=None)
df.head()

print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 3270


/tmp/ipykernel_188109/2226934891.py:1: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



#### Preprocesamiento del corpus de Bruno Mars

In [69]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []

for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

sentence_tokens[:3]

[['now',
  'greetings',
  'to',
  'the',
  'world',
  'standing',
  'at',
  'this',
  'liquor',
  'store'],
 ['whiskey', 'coming', 'through', 'my', 'pores'],
 ['feeling', 'like', 'i', 'run', 'this', 'whole', 'block']]

In [84]:
from gensim.models.callbacks import CallbackAny2Vec
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

####  Configuración y vocabulario del modelo Word2Vec

In [85]:
# Crearmos el modelo generador de vectores
# modelo Skipgram
w2v_model = Word2Vec(min_count=6,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=3,       # cant de palabras antes y desp de la predicha
                     vector_size=300,       # dimensionalidad de los vectores
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de docs en el corpus: 3270
Cantidad de words distintas en el corpus: 614


### 3 - Entrenar embeddings

In [72]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 236080.9375
Loss after epoch 1: 169850.59375
Loss after epoch 2: 168711.34375
Loss after epoch 3: 165889.75
Loss after epoch 4: 162720.125
Loss after epoch 5: 152928.0
Loss after epoch 6: 131406.25
Loss after epoch 7: 128045.25
Loss after epoch 8: 122721.625
Loss after epoch 9: 121018.25
Loss after epoch 10: 118915.5
Loss after epoch 11: 115525.75
Loss after epoch 12: 114036.625
Loss after epoch 13: 114323.0
Loss after epoch 14: 109931.75
Loss after epoch 15: 102697.0
Loss after epoch 16: 102263.5
Loss after epoch 17: 100955.75
Loss after epoch 18: 100374.0
Loss after epoch 19: 100131.5


(320726, 538000)

### 4 - Ensayas con palabras seleccionadas 

In [ ]:
# Palabras que MÁS se relacionan con getaway:
w2v_model.wv.most_similar(positive=["getaway"], topn=10)

[("there'll", 0.19892820715904236),
 ('things', 0.16908866167068481),
 ('da', 0.14479704201221466),
 ('nowhere', 0.14292305707931519),
 ('distance', 0.14131085574626923),
 ('greetings', 0.13557173311710358),
 ('tonight', 0.13217182457447052),
 ('than', 0.13138937950134277),
 ('blame', 0.12868192791938782),
 ('wrong', 0.12761081755161285)]

**Similitud con “getaway”**

El modelo asocia “getaway” con palabras que refuerzan la idea de escape o distancia romántica: aparecen términos como *distance*, *nowhere* y *greetings*, además de matices emocionales como *blame*, *wrong* y referencias nocturnas (*tonight*). Esto cuadra con las letras de Bruno Mars, donde “getaway” suele usarse en contextos de discusiones de pareja o planes para “escapar” juntos.


In [88]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["twenty"], topn=10)

[('said', 0.16283904016017914),
 ('his', 0.15440554916858673),
 ('own', 0.15256601572036743),
 ('estribillo', 0.15018446743488312),
 ('someone', 0.12427839636802673),
 ("that's", 0.12056832760572433),
 ('pop', 0.11863680928945541),
 ('making', 0.11392221599817276),
 ('that', 0.10987909138202667),
 ('many', 0.10931378602981567)]

**Palabras “menos relacionadas” con “twenty”**

Invertir el vector de “twenty” trae términos ligados a narrativas ajenas o a la industria musical: *said*, *his*, *own*, *someone*, *pop*, *making*, etc. El modelo aprende que “twenty” aparece en contextos específicos (probablemente referencias a edades o números), mientras que estas palabras se usan en otros escenarios (comentarios sobre “alguien”, procesos creativos, estribillos). Sirven como contraste: muestran qué vocabulario queda lejos del concepto de edad dentro del corpus de Bruno Mars.



In [90]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["girl"], topn=10)

[('mirror', 0.1961217224597931),
 ('nobody', 0.1735713928937912),
 ('has', 0.16421820223331451),
 ('not', 0.15566405653953552),
 ('sure', 0.14561836421489716),
 ('play', 0.14508208632469177),
 ('gimme', 0.13195320963859558),
 ('would', 0.12884579598903656),
 ('yeah', 0.1280348151922226),
 ('how', 0.11913199722766876)]

**Vecindario de “girl”**

Las palabras más cercanas a *girl* mezclan referencias románticas y de autoestima: *mirror*, *nobody*, *sure*, *play*, *gimme*, *yeah*. Reflejan letras donde se anima a la chica (“mirror”/“nobody has to know”), se habla de juegos/coqueteos (*play*, *gimme*) y se combinan con expresiones coloquiales (*yeah*, *how*). El modelo captura bien ese tono típico de Bruno Mars al dirigirse a una chica.


In [ ]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["nobody"], topn=10)

[('mirror', 0.1961217224597931),
 ('nobody', 0.1735713928937912),
 ('has', 0.16421820223331451),
 ('not', 0.15566405653953552),
 ('sure', 0.14561836421489716),
 ('play', 0.14508208632469177),
 ('gimme', 0.13195320963859558),
 ('would', 0.12884579598903656),
 ('yeah', 0.1280348151922226),
 ('how', 0.11913199722766876)]

**Vecindario de “nobody”**

El modelo ubica *nobody* cerca de palabras románticas o de empoderamiento: *girl*, *fine*, *give*, *forever*, además de expresiones coloquiales como *ooo* y referencias a juventud (*young*). En las letras de Bruno Mars, “nobody” suele aparecer cuando asegura que “nadie más” puede amar o cuidar como él, y los vecinos capturan justo esa mezcla de halagos y declaraciones de exclusividad.


In [99]:
# el método `get_vector` permite obtener los vectores:
vector_party = w2v_model.wv.get_vector("party")
print(vector_party)

[ 3.3327469e-03  1.5427971e-03  9.4840804e-04 -1.5575528e-04
  2.1733176e-03  1.1225402e-03  1.0065198e-03 -4.9335760e-04
 -2.5440033e-03 -5.1164109e-04  6.0100394e-04  3.0029833e-03
  3.0607248e-03  2.9130611e-03  1.1810938e-04  2.2352047e-03
  2.6088841e-03 -7.1613712e-04 -2.2100607e-05 -1.8344283e-04
 -1.4108964e-03  2.1110757e-03 -1.2345258e-03 -2.2260018e-03
  2.5964964e-03 -1.6600982e-03  2.1899224e-04  4.1409492e-04
  4.8350095e-05 -2.6095819e-03 -1.6532023e-04  7.2082639e-04
  2.2326214e-03  1.6180071e-03  3.2004504e-03 -1.0545651e-03
 -2.2522998e-03 -4.6394230e-04 -3.1576578e-03  2.3838400e-04
 -2.3849651e-03  3.6118428e-05 -8.1413426e-04  5.8174017e-04
  2.4718540e-03  1.2586761e-03  3.0926459e-03  2.0900341e-03
  1.1616766e-03  2.0014935e-03  2.2213967e-03  9.7710057e-04
  3.0019588e-03 -1.6017787e-03  2.2201450e-03 -1.8657991e-03
  1.6997290e-03 -1.4697194e-04  2.0053065e-03  2.2027481e-03
 -2.2958843e-03 -3.2900230e-03  2.5498648e-03  1.2062875e-03
 -7.5266045e-04 -2.66717

In [100]:
w2v_model.wv.most_similar(vector_party, topn=10)

[('party', 1.0),
 ('things', 0.19293759763240814),
 ('yesterday', 0.1816510707139969),
 ('creo', 0.1751469522714615),
 ('make', 0.15405318140983582),
 ('club', 0.14153331518173218),
 ('liquor', 0.1335722953081131),
 ('deserve', 0.13021107017993927),
 ('tryna', 0.1255696415901184),
 ('many', 0.11983267962932587)]

**Vecindario de “party”**

El modelo ubica *party* junto a vocabulario típico de noche y clubbing —*club*, *liquor*, *tryna*, *many*— y lo mezcla con términos que evocan recuerdos o consecuencias (*things*, *yesterday*). Esto refleja cómo Bruno Mars usa “party” tanto para describir el ambiente festivo como para contrastarlo con lo que pasa después (merecerlo, seguir adelante).


In [96]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["love"], topn=10)

[('myself', 0.18828529119491577),
 ('lay', 0.160945862531662),
 ('want', 0.15428900718688965),
 ('tomorrow', 0.14057257771492004),
 ('rain', 0.13834936916828156),
 ('ways', 0.1358347088098526),
 ('en', 0.12725508213043213),
 ('champagne', 0.12037444859743118),
 ('yo', 0.11811110377311707),
 ('somewhere', 0.11798980087041855)]

**Vecindario de “love”**

El embedding de *love* queda rodeado de palabras introspectivas y románticas: *myself*, *lay*, *want*, *tomorrow*, *rain*, *somewhere*, además de guiños glamorosos como *champagne* y expresiones coloquiales (*yo/en*). En las letras de Bruno Mars, “love” suele aparecer en escenarios íntimos (pensar en uno mismo, desear algo, reflexionar sobre el mañana) y en escenas nocturnas o melancólicas, por eso esas palabras terminan tan cercanas en el espacio vectorial.


### 5 - Visualizar agrupación de vectores

In [80]:
from sklearn.decomposition import IncrementalPCA
from sklearn.manifold import TSNE
import numpy as np

def reduce_dimensions(model, num_dimensions = 2 ):

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [81]:
# # Graficar los embedddings en 2D

import sys
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import nbformat, sys
print(nbformat.__version__)
print(sys.executable)

pio.renderers.default = "colab" if "google.colab" in sys.modules else "browser"

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS = 200
fig = px.scatter(x=vecs[:MAX_WORDS, 0], y=vecs[:MAX_WORDS, 1], text=labels[:MAX_WORDS])
fig.show(renderer="colab")

5.10.4
/home/seba/miniconda3/envs/llm/bin/python


### Conclusiones del mapa t‑SNE (Bruno Mars)

1. **Clúster en español aislado (x ≈ −40)**  
   Términos como `que`, `yo`, `estoy`, `pero`, `esta`, `ya` quedan juntos y muy lejos del núcleo en inglés. Son pocas palabras en español que aparecen en algunos coros; al compartir contexto y ser minoría, el modelo las separa claramente.

2. **Adjetivos y verbos de acción en el centro-derecha (x entre −10 y +20)**  
   Aquí se mezclan palabras como `beautiful`, `lonely`, `crazy`, `want`, `make`, `gonna`, `need`. Reflejan el eje lírico principal de Bruno Mars: adjetivos emotivos junto con verbos de promesa o deseo. Se percibe un gradiente: adjetivos y emociones hacia la parte superior/derecha, verbos y conectores más cerca del eje horizontal.

3. **Pronombres y marcadores narrativos (x < 0, y cercano a 0)**  
   `i'm`, `i've`, `she's`, `we`, `you`, `somewhere`, `there`, `been` forman una banda media-baja. El modelo los reconoce como similares entre sí pero distintos de los adjetivos fuertes; funcionan como “puentes” narrativos entre el clúster español y el núcleo principal en inglés.

En conjunto, el gráfico confirma que los embeddings capturan tanto los matices bilingües del corpus como la separación temática entre emociones, acciones y pronombres/conectores. ```


In [82]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

En la proyección 3D se distinguen capas que en 2D quedaban superpuestas:

- El clúster en español permanece aislado no solo en el eje X sino también ligeramente “por debajo” en Z, reforzando que esos términos comparten un contexto muy distinto del resto del corpus.
- Los verbos y conectores (`make`, `take`, `gonna`, `if`, `because`) forman una banda intermedia que envuelve al bloque central, lo que permite ver cómo ciertas palabras funcionales quedan “por fuera” del núcleo emocional.
- Los adjetivos/emociones (`lonely`, `crazy`, `beautiful`, `forever`) tienden a ubicarse en la parte superior/frontal del volumen, separados de los pronombres (`I`, `you`, `she’s`) que se agrupan cerca del plano central.

La vista tridimensional confirma tres familias semánticas: términos en español aislados, verbos/conectores rodeando el centro y adjetivos/emociones dominando la parte superior, lo que aporta más evidencia de cómo Bruno Mars distribuye su vocabulario según la función y el tono de las letras.


In [101]:
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

**Exportar a TensorBoard Projector**

Guardé todos los embeddings entrenados (`vectors.tsv`) y sus etiquetas (`labels.tsv`) para explorarlos en http://projector.tensorflow.org/. Esta herramienta permite navegar el espacio vectorial de Bruno Mars de forma interactiva: buscar palabras específicas, colorear clústeres y comparar distancias sin depender del t‑SNE local.
